In [2]:

from bs4 import BeautifulSoup
import bs4
import requests
import time
import pandas as pd
import random
import numpy as np

In [3]:
# A "data junior" search seems to give me jobs that I think are interesting: 
# https://ca.indeed.com/jobs?q=data%20junior&l=Canada
# When you go on the next page it:s
# https://ca.indeed.com/jobs?q=data+junior&l=Canada&start=10
# Then next page:
# https://ca.indeed.com/jobs?q=data+junior&l=Canada&start=20

# There only seems to be about 10 pages of results.
# If you try something like:
# https://ca.indeed.com/jobs?q=data+junior&l=Canada&start=1000
# It just give you the last page, so just check until the response stops changing?

# Also, you can use start=0 for the first page:
# https://ca.indeed.com/jobs?q=data+junior&l=Canada&start=0
# Is the same as
# https://ca.indeed.com/jobs?q=data%20junior&l=Canada

MAX_NUMBER_OF_PAGES_PER_SEARCH = 10 # testing

pages = []
for start in range(0, MAX_NUMBER_OF_PAGES_PER_SEARCH*10, 10):
    page = requests.get('https://ca.indeed.com/jobs?q=data%20junior&l=Canada&start=' + str(start))
    soup = BeautifulSoup(page.text, 'html.parser')
    pages.append(soup)
    time.sleep(random.randrange(3, 6) + random.random())

In [37]:
# from bs4 import BeautifulSoup
# from selenium import webdriver
# driver = webdriver.Chrome()
# driver.set_page_load_timeout(5)
# driver.get('https://ca.indeed.com/jobs?q=data%20junior&l=Canada&start=0')
# html = driver.page_source
# soup = BeautifulSoup(html)

# # check out the docs for the kinds of things you can do with 'find_all'
# # this (untested) snippet should find tags with a specific class ID
# # see: http://www.crummy.com/software/BeautifulSoup/bs4/doc/#searching-by-css-class
# for tag in soup.find_all("div", class_="jobsearch-SerpJobCard unifiedRow row result"):
#     print(tag.text)

In [38]:
import re

TEST_INPUT = """<ul style="list-style-type:circle;margin-top: 0px;margin-bottom: 0px;padding-left:20px;">\n<li style="margin-bottom:0px;">Interpret <b>data</b>, formulate reports, and make recommendations to the team.</li>\n<li>Remain fully informed on latest <b>data</b> trends, practice, and process.</li>\n</ul><ul style="list-style-type:circle;margin-top: 0px;margin-bottom: 0px;padding-left:20px;">\n<li style="margin-bottom:0px;">Interpret <b>data</b>, formulate reports, and make recommendations to the team.</li>\n<li>Remain fully informed on latest <b>data</b> trends, practice, and process.</li>\n</ul>"""

def clean_HTML(input: str) -> str:
    """I can't display HTML inside a cell in a DataFrame (it seems)"""
    # remove tags <somethign> and leave contents between tags
    out = re.sub("<.*?>", '', input)
    out = re.sub('\n', ' ', out)
    return out

print(clean_HTML(TEST_INPUT))

 Interpret data, formulate reports, and make recommendations to the team. Remain fully informed on latest data trends, practice, and process.  Interpret data, formulate reports, and make recommendations to the team. Remain fully informed on latest data trends, practice, and process. 


In [39]:
# When you search there's box cards for each posting, the class for the box is:
# class="jobsearch-SerpJobCard unifiedRow row result clickcard"
# that wasn't working for me though, so I changed it to 'class':"job_seen_beacon"

titles = pd.Series([], dtype='string')
companyNames = pd.Series([], dtype='string')
jobSnippets = pd.Series([], dtype='string')
for page in pages:
    jobs = page.find_all('div', attrs={'class':"job_seen_beacon"})
    for job in jobs:
        
        # Job Title
        title = job.find('h2', attrs={'class':'jobTitle'}).find('span', attrs={'title':True}).next 
        titles = titles.append(pd.Series([str(title)]))

        # Company Name
        companyName = job.find('span', attrs={'class':'companyName'})
        tempCompanyName = companyName
        companyName = companyName.find('a', attrs={'class':'turnstileLink companyOverviewLink'})
        if companyName is None: companyName = tempCompanyName
        companyName = companyName.next       
        companyNames = companyNames.append(pd.Series([str(companyName)]))
        
        jobSnippet = job.find('div', attrs={'class':'job-snippet'}).next
        jobSnippets = jobSnippets.append(pd.Series([str(clean_HTML(str(jobSnippet)))]))

pd.set_option('display.max_colwidth', None)
titles, companyNames, jobSnippets
job

<div class="job_seen_beacon"><table cellpadding="0" cellspacing="0" class="jobCard_mainContent" role="presentation"><tbody><tr><td class="resultContent"><div class="heading4 color-text-primary singleLineTitle tapItem-gutter"><h2 class="jobTitle jobTitle-color-purple"><span title="Data Steward I, ITSS Data Governance">Data Steward I, ITSS Data Governance</span></h2></div><div class="heading6 company_location tapItem-gutter"><pre><span class="companyName"><a class="turnstileLink companyOverviewLink" data-tn-element="companyName" href="/cmp/Td-Bank" rel="noopener" target="_blank">TD Bank</a></span><span class="ratingsDisplay withRatingLink"><a class="ratingLink" data-tn-variant="cmplinktst2" href="/cmp/Td-Bank/reviews" rel="noopener" target="_blank" title="TD Bank reviews"><span aria-label="3.8 of stars rating" class="ratingNumber" role="img"><span aria-hidden="true">3.8</span><svg aria-hidden="true" class="starIcon" fill="none" height="12" role="presentation" viewbox="0 0 16 16" width="1

In [40]:
pd.set_option('display.max_rows', 500)
assert len(titles) == len(companyNames) == len(jobSnippets)
pd.DataFrame({f'{titles=}'.split('=')[0]: titles, f'{companyNames=}'.split('=')[0]: companyNames, f'{jobSnippets=}'.split('=')[0]: jobSnippets})

,titles,companyNames,jobSnippets
0,Junior Data Analyst,O2E Brands,Passion for data visualization with Tableau. Experience blending data from disparate sources. Ensure consistent application of data governance initiatives.
0,Junior Data Analyst,OSL Retail Services Inc,Acquire data from primary or secondary data sources and maintain databases/data systems. Proven working experience as a data analyst or business data analyst.
0,Junior Data Analyst,Scandinavian Building Services,Previous experience with large scale data analysis or data reconciliations. Advanced level Microsoft Excel and data analytics skills.
0,"Product Manager, Junior - Global Data - Remote, CAN",Flight Centre Travel Group,"A background in data solution delivery, product management and Agile methodologies. Use your knowledge of reporting technologies to transform data into decision…"
0,Junior Data Analyst,FlightHub,Analyzing data sets and identifying trends and patterns; Our team will count on you to use data from various sources to identify issues as they happen and to…
0,Junior Research Analyst,Carleton University,?experience organizing and validating data. ?experience with data visualization and reporting. The ability to organize data accurately and prepare for reporting…
0,Jr. Data Analyst,Ratehub,Support in developing and maintaining our data model by understanding key data sources and relationships between them.
0,JUNIOR FINANCIAL ANALYST,Ministry of Colleges and Universities,Research and analyze statistical data from a variety of sources. You have well developed knowledge of financial information systems in order to obtain and…
0,Junior Data Engineer,Kora,Help maintain and update the company-level data warehouse to ensure data accuracy. The Data Engineer will support our financial department and data scientists…
0,Junior Machine Learning Scientist,Ecoation,"Analyzing customer horticultural data to provide meaningful insights. Occasional field visits to greenhouses to collect data, develop or test approaches."
